In [22]:
from fgn.completion.complete import acreate
from matrix_factory.chat_helpers import ok_models, instruct_models
import time
import openai
import anyio
from typing import Iterable, List
import itertools


# 1. Model Selection
def model_generator(models: List[str]):
    """
    A generator that yields models in a round-robin fashion using itertools.cycle.
    """
    return itertools.cycle(models)

# 2. Prompt Generation (could be expanded if there are more prompt types)
def construct_openai_prompt(base_prompt: str, item: str) -> str:
    return f"{base_prompt}: {item}"

# 3. OpenAI Call
async def call_openai(prompt: str, model_name: str, max_tokens: int = 50) -> str:
    return await acreate(prompt=prompt, model=model_name, max_tokens=max_tokens)


# 4. Timing and Logging
async def timed_openai_call(prompt: str, item: str, model_name: str, max_tokens: int = 50) -> str:
    start_time = time.time()
    response = await call_openai(prompt, model_name, max_tokens)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Generated response for \"{item[:30]}...\" using {model_name} in {elapsed_time:.2f} seconds.")
    return response

# 5. Concurrent Execution
async def prompt_map(base_prompt: str, iterable: Iterable[str], max_tokens: int = 50, model_list: List[str] = None) -> List[str]:
    model_gen = model_generator(model_list or instruct_models)
    responses = []

    async def generate_response(item: str) -> None:
        model_name = next(model_gen)
        prompt = construct_openai_prompt(base_prompt, item)
        response = await timed_openai_call(prompt, item, model_name, max_tokens)
        print(response)
        responses.append(response)

    async with anyio.create_task_group() as tg:
        for item in iterable:
            tg.start_soon(generate_response, item)

    return responses

# [Rest of your functions and example usage]

# Example models and usage:

example_prompt = "Translate the following text into French"
text_list = ["Hello, how are you?", "I love programming", "Thank you very much", "I am doing well", "I am doing poorly"]
start_time = time.time()
responses_best_models = await prompt_map(example_prompt, text_list, 50)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Responses: in {elapsed_time:.2f} seconds.", responses_best_models)


Generated response for "I am doing well..." using gpt-3.5-turbo-instruct-0914 in 0.34 seconds.
Je vais bien
Generated response for "I love programming..." using gpt-3.5-turbo-instruct-0914 in 0.34 seconds.
J'aime programmer
Generated response for "Thank you very much..." using gpt-3.5-turbo-instruct in 0.34 seconds.
Merci beaucoup
Generated response for "I am doing poorly..." using gpt-3.5-turbo-instruct in 0.34 seconds.
Je ne vais pas bien
Generated response for "Hello, how are you?..." using gpt-3.5-turbo-instruct in 0.34 seconds.
Bonjour, comment ça va ?
Responses: in 0.34 seconds. ['Je vais bien', "J'aime programmer", 'Merci beaucoup', 'Je ne vais pas bien', 'Bonjour, comment ça va ?']


In [26]:
async def prompt_filter(base_prompt: str, iterable: Iterable[str], model_list: List[str] = None, max_tokens: int = 50) -> List[str]:
    """
    This function takes a base prompt and filters an iterable based on responses from OpenAI.

    Args:
        base_prompt (str): The base prompt for generating boolean responses.
        iterable (iterable): An iterable (e.g., list, tuple) of strings to be filtered.
        model_list (List[str]): List of models to be used in round-robin fashion. If None, defaults to instruct_models.
        max_tokens (int): The maximum number of tokens in each response.

    Returns:
        list: A list of items from the iterable that pass the condition specified by the prompt.
    """

    responses = await prompt_map(base_prompt, iterable, max_tokens, model_list)
    results = [item for item, resp in zip(iterable, responses) if resp.strip().lower() == "true"]

    return results


filter_prompt = "Based on the following text, provide a boolean response ('true' or 'false'): "


statements = [
    "The sun is shining today.",
    "I failed the test.",
    "The flowers are blooming.",
    "The weather is gloomy.",
    "I aced the exam!"
]

filtered_statements = await prompt_filter(filter_prompt, statements)
print("Positive statements:", filtered_statements)



Generated response for "The sun is shining today...." using gpt-3.5-turbo-instruct in 0.24 seconds.
true
Generated response for "The flowers are blooming...." using gpt-3.5-turbo-instruct in 0.25 seconds.
true
Generated response for "I aced the exam!..." using gpt-3.5-turbo-instruct in 0.25 seconds.
true
Generated response for "The weather is gloomy...." using gpt-3.5-turbo-instruct-0914 in 0.25 seconds.
true
Generated response for "I failed the test...." using gpt-3.5-turbo-instruct-0914 in 0.27 seconds.
true
Positive statements: ['The sun is shining today.', 'I failed the test.', 'The flowers are blooming.', 'The weather is gloomy.', 'I aced the exam!']
